In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

from IPython.display import Markdown

In [2]:
from hybrid_search import hybrid_search

In [3]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
voyage_apikey = 'pa-g4x1yHL4Hv6qBjXcrT2Nhm5gG0srlDhVBp6Op7_IJBQ'


qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = ElasticsearchBM25()

Connecting to Elasticsearch at elasticsearch:9200


In [11]:
q = """
meeting notetakers
"""

In [12]:
import os
QDRANT_HOST = os.getenv('QDRANT_HOST')
QDRANT_PORT = os.getenv('QDRANT_PORT')

In [13]:
from qdrant_client import AsyncQdrantClient

In [14]:
r = await hybrid_search(q, qdrant_engine, es_engine)

In [15]:
import pandas as pd
df = pd.DataFrame(r['results']).sort_values('meeting_id', ascending=False)

In [32]:
def format_meeting_context(df, show_content=True, show_contextualized=True):
    unique_meetings = df['meeting_id'].unique()
    meeting_map = {mid: idx+1 for idx, mid in enumerate(unique_meetings)}
    meeting_map_reverse = {v:k for k,v in meeting_map.items()}
    
    # Sort by timestamp for meetings, then by formatted_time within meetings
    df = df.sort_values(['timestamp', 'formatted_time'])
    
    meetings = []
    for meeting_id, group in df.groupby('meeting_id'):
        int_meeting_id = meeting_map[meeting_id]
        timestamp = pd.to_datetime(group['timestamp'].iloc[0])
        date_header = f"## Meeting {int_meeting_id} - {timestamp.strftime('%B %d, %Y %H:%M')}"
        
        content_items = []
        for _, row in group.iterrows():
            time_prefix = f"[{row['formatted_time']}]" if row['formatted_time'] else ''
            if show_contextualized:
                content_items.append(f"- {time_prefix} {row['contextualized_content']}")
            if show_content:
                prefix = "  > " if show_contextualized else "- "
                content_items.append(f"{prefix}{row['content']}")
        
        content = "\n".join(content_items)
        meetings.append(f"{date_header}\n\n{content}\n")
    
    return "\n".join(meetings), meeting_map_reverse

In [33]:
input_context, meeting_id_map = format_meeting_context(df,show_contextualized=True)

In [34]:
display(Markdown(input_context))

## Meeting 1 - December 12, 2024 16:00

- [00:00] This chunk is part of a conversation between representatives of AI Marketing Directory and Dmitry Grankin, discussing their respective companies, recent changes, and the focus on content creation for founder-led businesses. It serves as an introduction to the dialogue.
  > AI Marketing Directory:  I guess while he does, yeah, well, it's nice to speak to you again after a little while.  It sounds like with both of us, others, Phil.  Hey, guys.  Hey, guys.
- [00:12] This chunk is part of a conversation between representatives of AI Marketing Directory and Dmitry Grankin, discussing the evolution of their respective companies and the changes they have made since their last interaction.
  > AI Marketing Directory:  So yeah, Phil and I started the company together.  We've made significant changes since we last spoke, which it sounds like you have as well, you know, which is what brought you to reach out to me in the first place.
- [00:56] This chunk is part of a conversation between representatives of AI Marketing Directory and Dmitry Grankin, discussing the company's strategic pivot towards creating high-quality content for founder-led companies, with an initial focus on Twitter for distribution.
  > AI Marketing Directory:  So yeah, I guess now instead of really targeting content on our own distribution, We are targeting founder-led companies and trying to create high-quality content by real people on our team for either the founders or their Brands socials, we're starting with Twitter for now because that's where we have the most success, you know actual documented success But eventually we'll be trying to get over to LinkedIn probably
- [02:03] This chunk is part of a conversation between representatives of AI Marketing Directory and Dmitry Grankin, discussing the company's pivot towards creating tailored content strategies for founder-led companies. It outlines their approach to understanding client products, analyzing market competition, and developing a collaborative content plan over a specified timeline.
  > AI Marketing Directory:  would be correct yeah we were pivoting right yeah um yeah so essentially as as ethan said we we We are going to like, well, what we've been doing with our clients is taking like a whole look at their product under completely understanding what our client's product is looking at competitive and what content is doing well in the marketplace.  And then just like taking all of that into account, like doing a deep dive into the target audience and then putting together a content content plan and executing on that content plan with our content team over like a 30 60 90 day period and obviously beyond that but that's that's what we would start with yeah and like I guess also the goal on top of what Phil said would be to like, you know, we want to actually learn, you know, everything there is to learn about our customers and do the best job we can.  The way we, you know, do that is, you know, just by making it more quality over quality, I guess you could say.  So like, yeah, you know, just really like making it a collaborative process, you know, And being able to share their ideas, but also just like really creating a specific catered plan 30 60 90. No, okay schedule so
- [04:11] This chunk occurs during a conversation between representatives of AI Marketing Directory and Dmitry Grankin, the founder of Vexa, where they discuss a shift in the marketing company's focus and express interest in collaborating on Vexa's content strategy.
  > AI Marketing Directory:  I guess the goal from our side was just to inform you that we're changing our direction and maybe this something that might be more interesting to you um than what we spoke about before um and yeah i mean we're happy to talk more on this call about vexa specifically i mean we could also schedule more calls in the future to do that whatever you prefer okay I love that
- [05:12] This chunk occurs during a conversation between the AI Marketing Directory and Dmitry Grankin, where they discuss their experiences with various meeting note-taking products, including Fireflies and Otter AI, as part of a broader dialogue about the functionality and usefulness of such tools in their work processes.
  > AI Marketing Directory:  have you been using such products I I have tested a couple of them yeah work wise or which ones I've tested Fireflies and then... What's the one that Nate uses for the meeting? I don't know which one he uses, but I've used Fireflies, and then let me see what else I've used.  I'm gonna try and look up the one he uses.  I have not actually stuck with one yet personally, but I've tried a couple of different ones.  I like use, yeah, I personally use it for myself rather than sending it to other- Oh, TLDV.  Okay.  in Otter AI is another one that I've tried.  Oh, that's the one, Otter AI, yeah.  Okay.
- [08:42] This chunk occurs during a conversation between the AI Marketing Directory and Dmitry Grankin, where they discuss the use of meeting note-taking tools. Dmitry shares his personal experience with summarizing calls for his older partner, highlighting the limited use he finds for the tool, which is primarily for recalling general ideas from meetings.
  > AI Marketing Directory:  literally only one specific i started doing it for one reason because our third partner um is an older guy and he uh he you know sometimes when i like talk to him about calls that i have a little summary so I literally you know if he asked me I'll pull it up but but that's pretty much the only use I found for it so far you've been reporting with was this  so you've been creating reports about your communities with with the other partner  no no so like i summarize my i don't use it for like you know sometimes You know, like I, I do it for, you know, I summarize like a call that I had, I'll put it, you know, in like a Google drive document and like, you know, if, if my, if the other partner asks us about it, I'll pull it up on my phone.  I'll just read it you know okay I don't do anything else with it after that yeah  basically you want to recall at least some general idea what was going on
- [09:55] This chunk occurs during a discussion between the AI Marketing Directory and Dmitry Grankin about the use and perceived value of meeting note-taking tools. The conversation reflects on their personal experiences with such tools, their current needs, and the potential benefits of using them in a business context.
  > AI Marketing Directory:  Yeah, at one point we got, and also to answer your question about our potential interests, we have our opinions about it, but if we had a much bigger company, maybe we'd be less reluctant to to getting it you know but um like like um uh i mean i guess what i'm trying to say is like you know like at one point i got like a free trial for it or something and that's why was using it i think you know um you know we're i guess it's just not something that we feel like we need right now and like that's sort of the stage run but like i definitely 100 see the value you know  okay okay but basically did you have any like you don't use it for a for a daily for for every meeting uh even though it's pretty easy to sit up and uh i don't yeah personally i don't but i i i mean i know a lot of people  that do i've met with a lot of people that do i know there's value there for sure okay
- [19:14] This chunk occurs towards the end of a conversation between representatives of AI Marketing Directory and Dmitry Grankin, the founder of Vexa. It focuses on discussing the next steps for collaboration, specifically regarding the crafting of a new landing page and how the AI Marketing Directory can support Vexa in growing its personal brand and content strategy.
  > AI Marketing Directory:  So I guess maybe the next step for us might be, you know, like i said we're crafting a new landing page but we could send you we pretty much have it all we know what we're what it's going what's going to be on it and everything so we could send you an email um you know with you know, all the, all the information, and we can maybe, you know, you can let us know how you feel about that, and I guess, you know, in light of your situation, you know, we'll, we'll, you know, we'll see, um, like, but, um, like, what we can deal with with respect to that um so does that sound good to you yeah sure um all right and then yeah sorry we're gonna say no i was gonna say yeah we could just send over an email with what our offer is and kind of how we think it would work well for BEXA and you personally, especially if you're trying to grow your personal brand along with BEXA.  Because obviously, that's going to help.  So yeah, we'll send over all the details, and then.  MARIUSZ GASIEWSKI, But can you share with me the idea?
- [22:22] This chunk is part of a conversation between the AI Marketing Directory and Dmitry Grankin, discussing the structure and frequency of their potential partnership, specifically focusing on content planning and collaboration for VEXA, a meeting note-taking tool.
  > AI Marketing Directory:  Yeah, we'll detail out what the partnership would even look like, cause we would wanna meet with you like every two weeks, get an idea of what's going on with VEXA and he, product updates, any thing that you want to push to these audiences that we're building up for you. Any like industry news that you want to touch on like those kind of things. And then, like come up with a content plan based on our conversations for like the next two weeks and then we just kind of do it all over again yeah I feel like as of May
- [24:47] This chunk is part of a conversation between the AI Marketing Directory and Dmitry Grankin, where they discuss the importance of maintaining constant communication with clients to effectively manage their personal brands and create tailored content that aligns with their goals and narratives.
  > AI Marketing Directory:  Yeah, that's why we would want to be in constant communication with anyone we're working with because we're not in the day-to-day.  So getting those updates makes it super simple.  And then also just like if we're managing, a personal brand, what are the types of content that they want created?  What are the topics that they want?  What are the stories they want to tell?
- [25:19] This chunk is part of a conversation between representatives of AI Marketing Directory and Dmitry Grankin, discussing their new approach to content creation for founder-led companies. It highlights their expertise in crafting viral content and leveraging an influencer network, emphasizing the importance of both content creation and distribution strategies in their marketing efforts.
  > AI Marketing Directory:  And we could just, we just have the knowledge and skill set to put that together in a way that has like viral potential yeah and two things i want to say uh echoing phil's point yeah like one that's like the common problem we're finding is like you know all these founders they have great ideas obviously for other starting companies.  But like Phil said, like, you know, we have the knowledge and expertise to not only put it together in a way that's well, but quickly.  And the second thing is like, you know, we're also gonna be utilizing our, influencer network that we've been aggregating.  Part of the reason we're targeting Twitter first and not LinkedIn right now is not just because we have probably more knowledge of Twitter, but we've been aggregating a huge influencer network for a long time.  know, um, you know, and we we pay for that exposure out of our pockets. So we take care of that, you know, so there are elements of distribution and elements of creation involved in it. So, you know, it's it's not


In [10]:
[v for v in meeting_id_map.values()]

NameError: name 'meeting_id_map' is not defined

In [36]:
from core import generic_call,system_msg,user_msg,count_tokens

In [37]:
input = count_tokens(input_context)

In [38]:
input

2517

In [39]:
from core import generic_call,system_msg,user_msg


In [40]:
system_prompt = """
you are helpful assistant that is answering user requests by interpreting  while interpreting the context
You have to rely on the context to answer the user request

User is Dmitry Grankin, Address his as YOU and distinguish them in in the provided context in in the response.

Always supply reference to specific meeting with [#meeting_id]

If appropriate, inlude qoutes with speaker reference

whenever you're responding consider everything you know about me in the memory to form a context of things that I would find interesting and where possible link back to those topics and include key terminologies and concepts that will help expand my knowledge along those areas.
"""




In [41]:

r = await generic_call([system_msg(system_prompt),
                        user_msg(f'context: {input_context}'),
                        user_msg(f'request: {q}')])
r



The representatives from the AI Marketing Directory are the individuals you were speaking with during the meeting on December 12, 2024. They are involved in discussing their company's strategic pivot towards creating high-quality content specifically for founder-led businesses, which aligns with your interests in content creation and marketing strategies for Vexa. 

In the meeting, they mentioned their focus on understanding client products, analyzing market competition, and developing tailored content strategies. They also highlighted their expertise in crafting viral content and leveraging an influencer network, particularly on Twitter, which could be beneficial for your personal brand and Vexa's marketing efforts. 

If you need more specific details about their roles or the company, feel free to ask!


"The representatives from the AI Marketing Directory are the individuals you were speaking with during the meeting on December 12, 2024. They are involved in discussing their company's strategic pivot towards creating high-quality content specifically for founder-led businesses, which aligns with your interests in content creation and marketing strategies for Vexa. \n\nIn the meeting, they mentioned their focus on understanding client products, analyzing market competition, and developing tailored content strategies. They also highlighted their expertise in crafting viral content and leveraging an influencer network, particularly on Twitter, which could be beneficial for your personal brand and Vexa's marketing efforts. \n\nIf you need more specific details about their roles or the company, feel free to ask!"

In [42]:
display(Markdown(r))

The representatives from the AI Marketing Directory are the individuals you were speaking with during the meeting on December 12, 2024. They are involved in discussing their company's strategic pivot towards creating high-quality content specifically for founder-led businesses, which aligns with your interests in content creation and marketing strategies for Vexa. 

In the meeting, they mentioned their focus on understanding client products, analyzing market competition, and developing tailored content strategies. They also highlighted their expertise in crafting viral content and leveraging an influencer network, particularly on Twitter, which could be beneficial for your personal brand and Vexa's marketing efforts. 

If you need more specific details about their roles or the company, feel free to ask!

In [37]:
def extract_tagged_content(text: str, tag: str) -> str:
    import re
    # Try exact match first
    pattern = f"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # If no exact match, try to find content after opening tag
    pattern = f"<{tag}>(.*?)(?:</?|$)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

# Example usage:
thinking = extract_tagged_content(r, "thinking")
answer = extract_tagged_content(r, "answer")

In [50]:
thinking

'Entities:\n- People: **Dmitriy Grankin**, **Lara Vargas**\n- Companies: None mentioned\n- Products: **VEXA**\n- Ideas: Engaging with **micro-influencers**, influencer outreach strategies, audience targeting, content creation, qualitative research\n- Action Points: \n  - Create a set of short descriptions about target audiences\n  - Identify and outreach to **micro-influencers**\n  - Develop a pipeline for influencer engagement\n  - Create YouTube queries for influencer research\n  - Analyze engagement rates of influencers\n  - Set up meetings with identified influencers'